<a href="https://colab.research.google.com/github/mkrsteska/BSA2020_Team_Tissot_Project_2/blob/master/code/BERT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT

In [0]:
!pip install bert-for-tf2

In [0]:
import pandas as pd
import numpy as np

import bert
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

#from keras.preprocessing.sequence import pad_sequences

#### BERT Tokenizer

In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/2", trainable = True)

vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

#### Load the data

In [0]:
df_train = pd.read_csv("https://raw.githubusercontent.com/mkrsteska/BSA2020_Team_Tissot_Project_2/master/data/train.csv")

In [0]:
X_train = df_train.text.values
y_train = df_train.target.values

In [0]:
MAX_LEN = 100

In [0]:
all_input_word_ids = np.array(all_input_word_ids)
all_masks = np.array(all_masks)
all_segment_ids = np.array(all_segment_ids)

## Model #1

In [0]:
# tokenize the tweets
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in X_train]
tokenized_tweets = [tweet[:MAX_LEN-2] for tweet in tokenized_tweets]

# add special tokens at the beginning and end of each tweet for BERT to work properly
tokenized_tweets = [["[CLS]"] + tweet + ["[SEP]"] for tweet in tokenized_tweets]

# convert the tokens to their index numbers in the BERT vocabulary
tokenized_tweets_ids = [tokenizer.convert_tokens_to_ids(tweet) for tweet in tokenized_tweets]

# pad the input tokens to max_len
all_input_word_ids = [tweet + [0]*(MAX_LEN-len(tweet)) for tweet in tokenized_tweets_ids]
# all_input_word_ids = pad_sequences(tokenized_tweets_ids, maxlen=MAX_LEN, dtype="long", padding="post")

# 1 - tokenized words, 0 - padded zeros
all_masks = [[1] * len(tweet) + [0] * (MAX_LEN - len(tweet)) for tweet in tokenized_tweets]

all_segment_ids = [[0] * MAX_LEN for tweet in tokenized_tweets]

In [0]:
input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

In [0]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

out = Dense(1, activation='sigmoid')(sequence_output[:, 0, :])
    
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

In [0]:
model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [13]:
model.fit(
    (all_input_word_ids, all_masks, all_segment_ids), y_train,
    validation_split=0.2,
    epochs=2,
    batch_size=4)

Epoch 1/2
1523/1523 [==============================] - 530s 348ms/step - loss: 0.4285 - accuracy: 0.8218 - val_loss: 0.3912 - val_accuracy: 0.8359
Epoch 2/2
1523/1523 [==============================] - 527s 346ms/step - loss: 0.2527 - accuracy: 0.9049 - val_loss: 0.3962 - val_accuracy: 0.8319


#### Tokenize the tweets from the test

In [0]:
df_test = pd.read_csv("https://raw.githubusercontent.com/mkrsteska/BSA2020_Team_Tissot_Project_2/master/data/test.csv")

In [0]:
X_test = df_test.text.values
ids = df_test['id'].to_numpy()

In [0]:
# tokenize the tweets
tokenized_tweets_test = [tokenizer.tokenize(tweet) for tweet in X_test]
tokenized_tweets_test = [tweet[:MAX_LEN-2] for tweet in tokenized_tweets_test]

# add special tokens at the beginning and end of each tweet for BERT to work properly
tokenized_tweets_test = [["[CLS]"] + tweet + ["[SEP]"] for tweet in tokenized_tweets_test]

# convert the tokens to their index numbers in the BERT vocabulary
tokenized_tweets_ids_test = [tokenizer.convert_tokens_to_ids(tweet) for tweet in tokenized_tweets_test]

# pad the input tokens to max_len
all_input_word_ids_test = [tweet + [0]*(MAX_LEN-len(tweet)) for tweet in tokenized_tweets_ids_test]
# all_input_word_ids = pad_sequences(tokenized_tweets_ids, maxlen=MAX_LEN, dtype="long", padding="post")

# 1 - tokenized words, 0 - padded zeros
all_masks_test = [[1] * len(tweet) + [0] * (MAX_LEN - len(tweet)) for tweet in tokenized_tweets_test]

all_segment_ids_test = [[0] * MAX_LEN for tweet in tokenized_tweets_test]

In [0]:
all_input_word_ids_test = np.array(all_input_word_ids_test)
all_masks_test = np.array(all_masks_test)
all_segment_ids_test = np.array(all_segment_ids_test)

**Create a submission file**

In [0]:
predictions = model.predict((all_input_word_ids_test, all_masks_test, all_segment_ids_test))

In [0]:
predictions_class = predictions.round().astype(int)[:, 0]

In [0]:
pd.DataFrame({'id': ids, 'target': predictions_class}).to_csv('16. Submission_BERT_2.csv', index=False)

**Score** 0.84253

## Model #2
preprocess tweets and different learning rate

Correct mislabeled samples

In [0]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
df_train.loc[df_train['id'].isin(ids_with_target_error),'target'] = 0

Preprocess tweets

In [0]:
from preprocess_tweets import preprocess_tweet_use

X_train = df_train.text.apply(preprocess_tweet_use)
y_train = df_train.target.values

In [0]:
# tokenize the tweets
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in X_train]
tokenized_tweets = [tweet[:MAX_LEN-2] for tweet in tokenized_tweets]

# add special tokens at the beginning and end of each tweet for BERT to work properly
tokenized_tweets = [["[CLS]"] + tweet + ["[SEP]"] for tweet in tokenized_tweets]

# convert the tokens to their index numbers in the BERT vocabulary
tokenized_tweets_ids = [tokenizer.convert_tokens_to_ids(tweet) for tweet in tokenized_tweets]

# pad the input tokens to max_len
all_input_word_ids = [tweet + [0]*(MAX_LEN-len(tweet)) for tweet in tokenized_tweets_ids]
# all_input_word_ids = pad_sequences(tokenized_tweets_ids, maxlen=MAX_LEN, dtype="long", padding="post")

# 1 - tokenized words, 0 - padded zeros
all_masks = [[1] * len(tweet) + [0] * (MAX_LEN - len(tweet)) for tweet in tokenized_tweets]

all_segment_ids = [[0] * MAX_LEN for tweet in tokenized_tweets]

In [0]:
all_input_word_ids = np.array(all_input_word_ids)
all_masks = np.array(all_masks)
all_segment_ids = np.array(all_segment_ids)

In [0]:
input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

In [0]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

out = Dense(1, activation='sigmoid')(sequence_output[:, 0, :])
    
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

In [0]:
model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
model.fit(
    (all_input_word_ids, all_masks, all_segment_ids), y_train,
    validation_split=0.2,
    epochs=3,
    batch_size=4)

Epoch 1/3
1523/1523 [==============================] - 555s 364ms/step - loss: 0.4334 - accuracy: 0.8108 - val_loss: 0.3840 - val_accuracy: 0.8378
Epoch 2/3
1523/1523 [==============================] - 554s 364ms/step - loss: 0.3272 - accuracy: 0.8686 - val_loss: 0.4151 - val_accuracy: 0.8437
Epoch 3/3
1523/1523 [==============================] - 549s 361ms/step - loss: 0.2479 - accuracy: 0.9033 - val_loss: 0.4584 - val_accuracy: 0.8326


#### Tokenize the tweets from the test data

In [0]:
df_test = pd.read_csv("https://raw.githubusercontent.com/mkrsteska/BSA2020_Team_Tissot_Project_2/master/data/test.csv")

In [0]:
X_test = df_test.text.apply(preprocess_tweet_use)
ids = df_test['id'].to_numpy()

In [0]:
# tokenize the tweets
tokenized_tweets_test = [tokenizer.tokenize(tweet) for tweet in X_test]
tokenized_tweets_test = [tweet[:MAX_LEN-2] for tweet in tokenized_tweets_test]

# add special tokens at the beginning and end of each tweet for BERT to work properly
tokenized_tweets_test = [["[CLS]"] + tweet + ["[SEP]"] for tweet in tokenized_tweets_test]

# convert the tokens to their index numbers in the BERT vocabulary
tokenized_tweets_ids_test = [tokenizer.convert_tokens_to_ids(tweet) for tweet in tokenized_tweets_test]

# pad the input tokens to max_len
all_input_word_ids_test = [tweet + [0]*(MAX_LEN-len(tweet)) for tweet in tokenized_tweets_ids_test]
# all_input_word_ids = pad_sequences(tokenized_tweets_ids, maxlen=MAX_LEN, dtype="long", padding="post")

# 1 - tokenized words, 0 - padded zeros
all_masks_test = [[1] * len(tweet) + [0] * (MAX_LEN - len(tweet)) for tweet in tokenized_tweets_test]

all_segment_ids_test = [[0] * MAX_LEN for tweet in tokenized_tweets_test]

In [0]:
all_input_word_ids_test = np.array(all_input_word_ids_test)
all_masks_test = np.array(all_masks_test)
all_segment_ids_test = np.array(all_segment_ids_test)

**Create a submission file**

In [0]:
predictions = model.predict((all_input_word_ids_test, all_masks_test, all_segment_ids_test))

In [0]:
predictions_class = predictions.round().astype(int)[:, 0]

In [0]:
pd.DataFrame({'id': ids, 'target': predictions_class}).to_csv('17. Submission_BERT_2.csv', index=False)

**Score** 0.84049